In [1]:
import numpy as np
import pandas as pd
from scipy.special import logit,expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [5]:
np.exp(0.07)

1.0725081812542165

In [2]:
Ia=np.array([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]])

In [3]:
S=np.array([1,1,1,1,1,1,1])

In [5]:
N=3000
P=7

In [6]:
data=pd.read_csv('wagedata.csv')

In [7]:
data.head()

,num,year,age,race,education,jobclass,health,health_ins,Y
0,1,2006,18,1,1,1,1,2,0.0
1,1,2004,24,1,4,2,2,2,0.0
2,1,2003,45,1,3,1,1,1,0.0
3,1,2003,43,3,4,2,2,1,0.0
4,1,2005,50,1,2,2,1,1,0.0


In [8]:
Y=np.array(data['Y'])
X=np.array(data.drop(['Y','num'],axis=1))

In [9]:
ss = StandardScaler()
xt=ss.fit_transform(X)

In [10]:
#np.array(np.where(Y==1)).shape
'''
SAM=np.array(data.loc[:,['num','age']])
asx=(SAM[:,1]-np.mean(SAM[:,1]))/(np.std(SAM[:,1]))
X=np.array([SAM[:,0],asx]).T
'''

"\nSAM=np.array(data.loc[:,['num','age']])\nasx=(SAM[:,1]-np.mean(SAM[:,1]))/(np.std(SAM[:,1]))\nX=np.array([SAM[:,0],asx]).T\n"

In [18]:
#E-step
def Lih(X,Y,beta):
    Ga=np.zeros((N,2))

    for i in range(N):#响应度
        if Y[i]==1:
            Ga[i,0]=0
            Ga[i,1]=1
        elif Y[i]==0:
            a=(expit(np.dot(X[i],np.dot(beta[0],Ia))))
            b=(1-a)*(1-expit(np.dot(X[i],beta[1])))
            Ga[i,0]=a/(a+b)
            Ga[i,1]=1-Ga[i,0]

    return Ga


#M-step
def Q(X,Y,beta):
    Ga=Lih(X,Y,beta)
    A=np.zeros(N)
    for i in range(N):
        pi=expit(np.dot(np.dot(beta[0],Ia),X[i]))
        phi=expit(np.dot(beta[1],X[i]))
        if Y[i]==0:
            A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
        else:
            A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
        #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
    Q=np.sum(A)
    return Q

def grad(X,Y,Ga,beta):
    A=np.zeros((N,P))
    B=np.zeros((N,P))
    for i in range(N):
        A[i]=((Ga[i,0]*(1-Y[i]))*(1-expit(np.dot(np.dot(beta[0],Ia),X[i])))-(Ga[i,1]*expit(np.dot(np.dot(beta[0],Ia),X[i]))))*np.dot(X[i],Ia)
        B[i]=(Ga[i,1]*(Y[i]-expit(np.dot(beta[1],X[i]))))*X[i]
    grad1=np.sum(A,axis=0)
    grad2=np.sum(B,axis=0)
    grad=np.array([grad1,grad2])
    return grad


#EM
def dgra(X,Y,ibeta,n=1000,t=0.01,xim=0.5,yim=0.3):#梯度下降（回溯梯度下降法）xim为步长缩小倍数

    for i in range(n):#M-step
        lik1=Q(X,Y,ibeta)
        ibeta1=ibeta
        L=Lih(X,Y,ibeta)#E-step
        ibeta=ibeta+t*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[np.dot(ibeta[0],Ia),ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        print('i=',i,'b=',b,'\tstep=',t)
        print('Q=',Q(X,Y,ibeta))
        if b>=yim*t*c :
            t=xim*t
        else:
            break
    return ibeta


#def dgra1(X,Y,ibeta,n=1000,a=0.01,yim=0.2,zim=0.001):
    
    for i in range(n):
        lik1=Q(X,Y,ibeta)
        L=Lih(X,ibeta)#E-step
        ibeta=ibeta+a*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        if b>=yim*a*c:
            a=0.5*a
        if c<=zim:
            break
    return ibeta

In [44]:
beta01=[0,0,4,4,0,0,0]
beta02=[1,2,6,5,1,1,1]
beta0=np.array([beta01,beta02])
Q(xt,Y,beta0)

-1042.728803682324

In [45]:
EMbeta=dgra(xt,Y,beta0)

i= 0 b= 29.807687830382747 	step= 0.01
Q= -1012.9211158519413
i= 1 b= 1.916393263685336 	step= 0.005
Q= -1011.004722588256
i= 2 b= -0.5068593832130546 	step= 0.0025
Q= -1011.511581971469


In [46]:
EMbeta

array([[0.        , 0.        , 3.74765835, 3.27689726, 0.        ,
        0.        , 0.        ],
       [0.72189356, 1.20826403, 6.3329544 , 5.13214686, 0.5354318 ,
        0.73186409, 0.793924  ]])